# Extracting and transforming data

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

### Positional and labeled indexing

In [3]:
election = pd.read_csv('pennsylvania2012_turnout.csv', index_col='county')
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118


In [4]:
x,y=4,4
election.iloc[x, y] == election.loc['Bedford', 'winner']

True

### Indexing and column rearrangement

In [5]:
results = election[['winner', 'total', 'voters']]
results.head()

,winner,total,voters
county,,,
Adams,Romney,41973,61156
Allegheny,Obama,614671,924351
Armstrong,Romney,28322,42147
Beaver,Romney,80015,115157
Bedford,Romney,21444,32189


### Slicing rows

In [6]:
slice_counties = election.loc['Perry':'Potter']
slice_counties.head()

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Perry,PA,18240,29.769737,68.591009,Romney,27245,66.948064,38.821272
Philadelphia,PA,653598,85.224251,14.051451,Obama,1099197,59.461407,71.172800
Pike,PA,23164,43.904334,54.882576,Romney,41840,55.363289,10.978242
Potter,PA,7205,26.259542,72.158223,Romney,10913,66.022175,45.898681


In [7]:
# Slicing in reverse order
slice_counties_rev = election.loc['Potter':'Perry':-1]
slice_counties_rev.head()

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Potter,PA,7205,26.259542,72.158223,Romney,10913,66.022175,45.898681
Pike,PA,23164,43.904334,54.882576,Romney,41840,55.363289,10.978242
Philadelphia,PA,653598,85.224251,14.051451,Obama,1099197,59.461407,71.172800
Perry,PA,18240,29.769737,68.591009,Romney,27245,66.948064,38.821272


### Slicing columns

In [8]:
left_columns = election.loc[:,:'Obama']
middle_columns = election.loc[:,'Obama':'winner']
right_columns = election.loc[:,'Romney':]
right_columns.head()

,Romney,winner,voters,turnout,margin
county,,,,,
Adams,63.112001,Romney,61156,68.632677,27.629667
Allegheny,42.185820,Obama,924351,66.497575,14.454399
Armstrong,67.901278,Romney,42147,67.198140,37.204293
Beaver,52.637630,Romney,115157,69.483401,6.605012
Bedford,76.986570,Romney,32189,66.619031,54.929118


### Subselecting DataFrames with lists

In [9]:
rows = ['Philadelphia', 'Centre', 'Fulton']
cols = ['winner', 'Obama', 'Romney']
three_counties = election.loc[rows,cols]
three_counties

,winner,Obama,Romney
county,,,
Philadelphia,Obama,85.224251,14.051451
Centre,Romney,48.948416,48.977486
Fulton,Romney,21.096291,77.748861


### Thresholding data

In [10]:
high_turnout = election[election.turnout > 70]
high_turnout.head()

,state,total,Obama,Romney,winner,voters,turnout,margin
county,,,,,,,,
Bucks,PA,319407,49.966970,48.801686,Obama,435606,73.324748,1.165284
Butler,PA,88924,31.920516,66.816607,Romney,122762,72.436096,34.896091
Chester,PA,248295,49.228539,49.650617,Romney,337822,73.498766,0.422079
Forest,PA,2308,38.734835,59.835355,Romney,3232,71.410891,21.100520
Franklin,PA,62802,30.110506,68.583803,Romney,87406,71.850903,38.473297


### Filtering columns using other columns

In [11]:
too_close = election.margin < 1
# This selection is preferd over election.winner[too_close] = np.nan
election.loc[too_close, 'winner'] = np.nan
election.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, Adams to York
Data columns (total 8 columns):
state      67 non-null object
total      67 non-null int64
Obama      67 non-null float64
Romney     67 non-null float64
winner     64 non-null object
voters     67 non-null int64
turnout    67 non-null float64
margin     67 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 7.2+ KB


### Filtering using NaN

In [12]:
titanic = pd.read_csv('titanic.csv')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB


In [13]:
sub_select = titanic[['age', 'cabin']]
sub_select.shape

(1309, 2)

In [14]:
sub_select.dropna(how='any').shape

(272, 2)

In [15]:
sub_select.dropna(how='all').shape

(1069, 2)

In [16]:
titanic.dropna(thresh=1000, axis='columns').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
pclass      1309 non-null int64
survived    1309 non-null int64
name        1309 non-null object
sex         1309 non-null object
age         1046 non-null float64
sibsp       1309 non-null int64
parch       1309 non-null int64
ticket      1309 non-null object
fare        1308 non-null float64
embarked    1307 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 102.3+ KB


### Using apply() to transform a column

In [17]:
weather = pd.read_csv('pittsburgh2013.csv')
weather.head()

,Date,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,Mean Dew PointF,Min DewpointF,Max Humidity,Mean Humidity,Min Humidity,...,Max VisibilityMiles,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
0,2013-1-1,32,28,21,30,27,16,100,89,77,...,10,6,2,10,8,NaN,0.00,8,Snow,277
1,2013-1-2,25,21,17,14,12,10,77,67,55,...,10,10,10,14,5,NaN,0.00,4,NaN,272
2,2013-1-3,32,24,16,19,15,9,77,67,56,...,10,10,10,17,8,26.0,0.00,3,NaN,229
3,2013-1-4,30,28,27,21,19,17,75,68,59,...,10,10,6,23,16,32.0,0.00,4,NaN,250
4,2013-1-5,34,30,25,23,20,16,75,68,61,...,10,10,10,16,10,23.0,0.21,5,NaN,221


In [18]:
def to_celsius(F):
    return 5/9*(F-32)

In [19]:
weather_c = weather[['Mean TemperatureF', 'Mean Dew PointF']].apply(to_celsius)

In [20]:
weather_c.columns = ['Mean TemperatureC', 'Mean Dew PointC']
weather_c.head()

,Mean TemperatureC,Mean Dew PointC
0,-2.222222,-2.777778
1,-6.111111,-11.111111
2,-4.444444,-9.444444
3,-2.222222,-7.222222
4,-1.111111,-6.666667


### Using .map() with a dictionary

In [21]:
red_vs_blue = {'Obama':'blue', 'Romney':'red'}
election['color'] = election['winner'].map(red_vs_blue)
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin,color
county,,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667,red
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399,blue
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293,red
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012,red
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118,red


### Using vectorized functions

In [22]:
turnout_zscore = zscore(election['turnout'])
type(turnout_zscore)

numpy.ndarray

In [23]:
election['turnout_zscore'] = turnout_zscore
election.head()

,state,total,Obama,Romney,winner,voters,turnout,margin,color,turnout_zscore
county,,,,,,,,,,
Adams,PA,41973,35.482334,63.112001,Romney,61156,68.632677,27.629667,red,0.853734
Allegheny,PA,614671,56.640219,42.185820,Obama,924351,66.497575,14.454399,blue,0.439846
Armstrong,PA,28322,30.696985,67.901278,Romney,42147,67.198140,37.204293,red,0.575650
Beaver,PA,80015,46.032619,52.637630,Romney,115157,69.483401,6.605012,red,1.018647
Bedford,PA,21444,22.057452,76.986570,Romney,32189,66.619031,54.929118,red,0.463391
